<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/quantconnect_lightgbm_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Fine-Tuned LightGBM Walkforward Strategy for QuantConnect
from AlgorithmImports import *
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from datetime import timedelta

class LightGBMWalkforward(QCAlgorithm):
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2024, 1, 1)
        self.SetCash(100000)

        self.symbol = self.AddEquity("BRK.B", Resolution.Hour).Symbol
        self.lookback = 500
        self.retrain_interval_days = 60
        self.last_training_time = self.Time - timedelta(days=self.retrain_interval_days)
        self.last_trade_time = self.Time - timedelta(hours=1)
        self.trade_interval = timedelta(hours=1)
        self.train_model_ready = False
        self.last_action = 0
        self.stop_pct = 0.02
        self.trailing_stop_price = None

        self.verbose = False

        self.features = [
            "SMA_50", "RSI", "MACD", "ATR", "Momentum", "%B"
        ]

        self.SetWarmUp(self.lookback, Resolution.Hour)
        self.SetBrokerageModel(BrokerageName.InteractiveBrokersBrokerage, AccountType.Margin)
        self.SetSecurityInitializer(lambda security: security.SetSlippageModel(ConstantSlippageModel(0.01)))

        self.Schedule.On(self.DateRules.EveryDay(self.symbol),
                         self.TimeRules.Every(timedelta(hours=1)),
                         self.TradeLogic)

    def OnData(self, data):
        if self.IsWarmingUp:
            return

        if (self.Time - self.last_training_time).days < self.retrain_interval_days:
            return

        history = self.History(self.symbol, self.lookback, Resolution.Hour).dropna()
        if history.empty:
            return

        df = history.reset_index().rename(columns={
            "time": "Datetime",
            "close": "Close",
            "high": "High",
            "low": "Low",
            "volume": "Volume"
        })

        df = self.AddFeatures(df)
        df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
        df.dropna(inplace=True)

        if len(df) < 100:
            return

        X = df[self.features]
        y = df['Target']

        self.scaler = MinMaxScaler()
        X_scaled = self.scaler.fit_transform(X)

        self.model = LGBMClassifier(n_estimators=120, max_depth=3, learning_rate=0.05)
        self.model.fit(X_scaled, y)

        self.train_model_ready = True
        self.last_training_time = self.Time
        self.Debug(f"Model retrained on {len(df)} samples")

    def AddFeatures(self, df):
        df['SMA_50'] = df['Close'].rolling(window=50).mean()

        delta = df['Close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=14).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
        rs = gain / (loss + 1e-6)
        df['RSI'] = 100 - (100 / (1 + rs))

        df['MACD'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
        df['ATR'] = df['High'].rolling(window=14).max() - df['Low'].rolling(window=14).min()
        df['Momentum'] = df['Close'] - df['Close'].shift(10)
        df['%B'] = (df['Close'] - df['Close'].rolling(20).mean()) / (2 * df['Close'].rolling(20).std())

        df.dropna(inplace=True)
        return df

    def TradeLogic(self):
        if not self.train_model_ready or self.IsWarmingUp:
            return

        if self.Time - self.last_trade_time < self.trade_interval:
            return

        history = self.History(self.symbol, 50, Resolution.Hour).dropna()
        if history.empty:
            return

        df = history.reset_index().rename(columns={
            "time": "Datetime",
            "close": "Close",
            "high": "High",
            "low": "Low",
            "volume": "Volume"
        })

        df = self.AddFeatures(df)
        if df.empty:
            return

        latest = df.iloc[-1:]
        price = latest["Close"].values[0]
        atr = latest["ATR"].values[0]
        sma = latest["SMA_50"].values[0]
        rsi = latest["RSI"].values[0]

        if atr / price < 0.008:
            return

        X_live = self.scaler.transform(latest[self.features])
        proba = self.model.predict_proba(X_live)[0][1]

        if self.last_action == 1 and self.trailing_stop_price:
            if price < self.trailing_stop_price:
                self.Liquidate(self.symbol)
                self.last_action = -1
                self.trailing_stop_price = None
                self.Debug(f"Trailing stop hit @ {price:.2f}")
                return
            else:
                self.trailing_stop_price = max(self.trailing_stop_price, price * (1 - self.stop_pct))

        if 0.48 < proba < 0.52:
            return

        if proba > 0.7 and self.last_action != 1 and price > sma and rsi > 50:
            size = round(min(0.5, max(0.1, (proba - 0.5) * 1.5)), 2)
            self.SetHoldings(self.symbol, size)
            self.trailing_stop_price = price * (1 - self.stop_pct)
            self.last_action = 1
            self.last_trade_time = self.Time
            self.Debug(f"BUY @ {price:.2f} — Prob: {proba:.2f} — Size: {size:.2f}")

        elif proba < 0.3 and self.last_action == 1:
            self.Liquidate(self.symbol)
            self.last_action = -1
            self.trailing_stop_price = None
            self.last_trade_time = self.Time
            self.Debug(f"SELL @ {price:.2f} — Prob: {proba:.2f}")
